### Data Manipulation

To start, we import `torch`. Note that though it's called PyTorch, we should
import `torch` instead of `pytorch`.

In [1]:
import torch

A tensor represents a (possibly multi-dimensional) array of numerical values. We can access a tensor's *shape*.

In [2]:
x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [3]:
x.shape

torch.Size([12])

In [4]:
x.numel()

12

We can reshape the array, e.g. explicitly via

In [5]:
X = x.reshape(3, 4)
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

We can automatically have the system infer shapes, by filling in `-1`, e.g. in 
`x.reshape(-1, 4)` or `x.reshape(3, -1)`.

To initialize the tensors explicitly, e.g. to $0$ or $1$ we can use

In [6]:
torch.zeros((2, 3, 4))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [7]:
torch.ones((2, 3, 4))

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

To generate a random matrix with Gaussian entries we can use `randn`.

In [8]:
torch.randn(3, 4)

tensor([[ 0.2136, -1.1994,  0.7703, -0.7753],
        [ 1.3893,  0.8534,  0.6527, -0.0026],
        [-0.0722, -0.1957,  0.7296,  1.6990]])

We can also generate tensors explicitly.

In [9]:
torch.tensor([[2, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])

tensor([[2, 1, 4, 3],
        [1, 2, 3, 4],
        [4, 3, 2, 1]])

### Operations

The common standard arithmetic operators
(`+`, `-`, `*`, `/`, and `**`)
have all been *lifted* to elementwise operations.

In [10]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x * y, x / y, x ** y  # The ** operator is exponentiation

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]))

Many more operations can be applied elementwise, e.g. `exp`.

In [11]:
torch.exp(x)

tensor([2.7183e+00, 7.3891e+00, 5.4598e+01, 2.9810e+03])

We can also *concatenate* multiple tensors together,
stacking them end-to-end to form a larger tensor.

In [12]:
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

Sometimes, we want to construct a binary tensor via *logical statements*.
Take `X == Y` as an example.

In [13]:
X == Y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

Summing all the elements in the tensor yields a tensor with only one element.

In [14]:
X.sum()

tensor(66.)

### Broadcasting Mechanism

Even when shapes differ, we can still perform elementwise operations
by invoking the *broadcasting mechanism*.

In [15]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

Since `a` and `b` are $3\times1$ and $1\times2$ matrices their shapes do not match up if we want to add them. We can add them nonetheless.

In [16]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

### Indexing and Slicing

`[-1]` selects the last element and `[1:3]`
selects the second and the third elements.

In [17]:
X[-1], X[1:3]

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]))

Beyond reading, we can also write elements of a matrix by specifying indices.

In [18]:
X[1, 2] = 9
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  9.,  7.],
        [ 8.,  9., 10., 11.]])

If we want to assign multiple elements the same value,
we simply index all of them and then assign them the value.

In [19]:
X[0:2, :] = 12
X

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]])

## Saving Memory

Running operations can cause new memory to be
allocated to host results.
For example, if we write `Y = X + Y`,
we will dereference the tensor that `Y` used to point to
and instead point `Y` at the newly allocated memory.

In [20]:
before = id(Y)
Y = Y + X
id(Y) == before

False

We want to do things in place to save memory. This can be done via `Y[:] = <expression>`.

In [21]:
Z = torch.zeros_like(Y)
print('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))

id(Z): 139663306084224
id(Z): 139663306084224


If the value of `X` is not reused in subsequent computations,
we can also use `X[:] = X + Y` or `X += Y`
to reduce the memory overhead of the operation.

In [22]:
before = id(X)
X += Y
id(X) == before

True

## Conversion to Other Python Objects

Converting to a NumPy tensor, or vice versa, is easy.
The converted result does not share memory.

Caution - when you perform operations on the CPU or on GPUs,
you do not want to halt computation.

In [23]:
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)

To convert a size-1 tensor to a Python scalar,
we can invoke the `item` function or Python's built-in functions.

In [24]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)